#MY FIRST-EVER HACKATHON
ORIGINAL NOTEBOOK



IMPORT ALL ESSENTIAL LIBRARIES

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import sklearn
import math
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor, XGBRFRegressor
from xgboost import plot_tree, plot_importance
from sklearn.ensemble import AdaBoostRegressor
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split

GATHER THE DATASETS

In [2]:
train_data = pd.read_csv('Train_Data.csv')
test_data = pd.read_csv('Test_Data.csv')

In [3]:
train_data.head()

,date,campaign,adgroup,ad,impressions,clicks,cost,conversions,revenue
0,01-08-2020,campaign 1,adgroup 1,ad 1,24,6,0.08,0,0.00
1,01-08-2020,campaign 1,adgroup 2,ad 1,1,0,0.00,0,0.00
2,01-08-2020,campaign 1,adgroup 3,ad 1,13,4,0.04,0,0.00
3,01-08-2020,campaign 1,adgroup 4,ad 1,5,4,0.08,0,0.00
4,01-08-2020,campaign 1,adgroup 1,ad 2,247,126,1.29,4,925.71


In [4]:
test_data.head()

,date,campaign,adgroup,ad,cost,impressions,clicks,conversions
0,01-03-2021,campaign 1,adgroup 1,ad 1,0.58,121,49,1
1,01-03-2021,campaign 1,adgroup 3,ad 1,0.17,22,12,0
2,01-03-2021,campaign 1,adgroup 4,ad 1,0.05,5,3,0
3,01-03-2021,campaign 1,adgroup 2,ad 1,0.01,2,1,0
4,01-03-2021,campaign 1,adgroup 2,ad 2,0.01,3,1,0


DATA PREPROCESSING

In [5]:
# Change the date format to day/month/year format
train_data[["day", "month", "year"]] =train_data["date"].str.split("-", expand = True)
train_data = train_data[['day', 'month', 'year', 'campaign', 'adgroup', 'ad', 'impressions', 'clicks', 'cost', 'conversions', 'revenue']]
train_data

,day,month,year,campaign,adgroup,ad,impressions,clicks,cost,conversions,revenue
0,01,08,2020,campaign 1,adgroup 1,ad 1,24,6,0.08,0,0.00
1,01,08,2020,campaign 1,adgroup 2,ad 1,1,0,0.00,0,0.00
2,01,08,2020,campaign 1,adgroup 3,ad 1,13,4,0.04,0,0.00
3,01,08,2020,campaign 1,adgroup 4,ad 1,5,4,0.08,0,0.00
4,01,08,2020,campaign 1,adgroup 1,ad 2,247,126,1.29,4,925.71
...,...,...,...,...,...,...,...,...,...,...,...
4566,28,02,2021,campaign 1,adgroup 3,ad 55,19,6,0.07,0,0.00
4567,28,02,2021,campaign 1,adgroup 4,ad 55,1,0,0.00,0,0.00
4568,28,02,2021,campaign 1,adgroup 3,ad 56,16,8,0.12,1,93.20
4569,28,02,2021,campaign 1,adgroup 1,ad 56,37,13,0.23,0,0.00


In [6]:
# Change the date format to day/month/year format for the test data too 
test_data[["day", "month", "year"]] =test_data["date"].str.split("-", expand = True)
test_data = test_data[['day', 'month', 'year', 'campaign', 'adgroup', 'ad', 'impressions', 'clicks', 'cost', 'conversions']]
test_data

,day,month,year,campaign,adgroup,ad,impressions,clicks,cost,conversions
0,01,03,2021,campaign 1,adgroup 1,ad 1,121,49,0.58,1
1,01,03,2021,campaign 1,adgroup 3,ad 1,22,12,0.17,0
2,01,03,2021,campaign 1,adgroup 4,ad 1,5,3,0.05,0
3,01,03,2021,campaign 1,adgroup 2,ad 1,2,1,0.01,0
4,01,03,2021,campaign 1,adgroup 2,ad 2,3,1,0.01,0
...,...,...,...,...,...,...,...,...,...,...
313,15,03,2021,campaign 1,adgroup 4,ad 55,1,1,0.01,0
314,15,03,2021,campaign 1,adgroup 3,ad 55,5,1,0.01,0
315,15,03,2021,campaign 1,adgroup 1,ad 55,25,11,0.14,0
316,15,03,2021,campaign 1,adgroup 3,ad 56,9,4,0.06,0


In [7]:
# See the types of ads and adgroups
print(train_data.ad.unique())
print(train_data.adgroup.unique())

['ad 1' 'ad 2' 'ad 3' 'ad 4' 'ad 5' 'ad 6' 'ad 7' 'ad 8' 'ad 9' 'ad 10'
 'ad 11' 'ad 12' 'ad 13' 'ad 14' 'ad 15' 'ad 16' 'ad 17' 'ad 18' 'ad 19'
 'ad 20' 'ad 21' 'ad 22' 'ad 23' 'ad 24' 'ad 25' 'ad 26' 'ad 27' 'ad 28'
 'ad 29' 'ad 30' 'ad 31' 'ad 32' 'ad 33' 'ad 34' 'ad 35' 'ad 36' 'ad 37'
 'ad 38' 'ad 39' 'ad 40' 'ad 41' 'ad 42' 'ad 43' 'ad 44' 'ad 45' 'ad 46'
 'ad 47' 'ad 48' 'ad 49' 'ad 50' 'ad 51' 'ad 52' 'ad 53' 'ad 54' 'ad 55'
 'ad 56' 'ad 57' 'ad 58' 'ad 59' 'ad 60' 'ad 61' 'ad 62' 'ad 63' 'ad 64'
 'ad 65' 'ad 66' 'ad 67' 'ad 68' 'ad 69' 'ad 70']
['adgroup 1' 'adgroup 2' 'adgroup 3' 'adgroup 4']


In [8]:
#Change the adgroups and ads to labels using LabelEncoder ==> adgroup1 = 0, adgroup2 = 1...
le = LabelEncoder()
train_data['adgroup'] = le.fit_transform(train_data['adgroup']) 
train_data['ad'] = le.fit_transform(train_data['ad']) 
train_data['campaign'] = le.fit_transform(train_data['campaign']) 
train_data.head()

,day,month,year,campaign,adgroup,ad,impressions,clicks,cost,conversions,revenue
0,01,08,2020,0,0,0,24,6,0.08,0,0.00
1,01,08,2020,0,1,0,1,0,0.00,0,0.00
2,01,08,2020,0,2,0,13,4,0.04,0,0.00
3,01,08,2020,0,3,0,5,4,0.08,0,0.00
4,01,08,2020,0,0,11,247,126,1.29,4,925.71


In [9]:
# Check for any null values
train_data.isna().sum()

day            0
month          0
year           0
campaign       0
adgroup        0
ad             0
impressions    0
clicks         0
cost           0
conversions    0
revenue        0
dtype: int64

In [10]:
#Change the adgroups and ads to labels using LabelEncoder in the test data too
le = LabelEncoder()
test_data['adgroup'] = le.fit_transform(test_data['adgroup']) 
test_data['ad'] = le.fit_transform(test_data['ad']) 
test_data['campaign'] = le.fit_transform(test_data['campaign']) 
test_data

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row

,day,month,year,campaign,adgroup,ad,impressions,clicks,cost,conversions
0,01,03,2021,0,0,0,121,49,0.58,1
1,01,03,2021,0,2,0,22,12,0.17,0
2,01,03,2021,0,3,0,5,3,0.05,0
3,01,03,2021,0,1,0,2,1,0.01,0
4,01,03,2021,0,1,2,3,1,0.01,0
...,...,...,...,...,...,...,...,...,...,...
313,15,03,2021,0,3,6,1,1,0.01,0
314,15,03,2021,0,2,6,5,1,0.01,0
315,15,03,2021,0,0,6,25,11,0.14,0
316,15,03,2021,0,2,7,9,4,0.06,0


In [11]:
# Get X_train and y_train
X_train = train_data.drop(['revenue'], axis=1)
y_train = train_data.revenue
print(X_train)
print(y_train)

     day month  year  campaign  adgroup  ad  impressions  clicks  cost  \
0     01    08  2020         0        0   0           24       6  0.08   
1     01    08  2020         0        1   0            1       0  0.00   
2     01    08  2020         0        2   0           13       4  0.04   
3     01    08  2020         0        3   0            5       4  0.08   
4     01    08  2020         0        0  11          247     126  1.29   
...   ..   ...   ...       ...      ...  ..          ...     ...   ...   
4566  28    02  2021         0        2  50           19       6  0.07   
4567  28    02  2021         0        3  50            1       0  0.00   
4568  28    02  2021         0        2  51           16       8  0.12   
4569  28    02  2021         0        0  51           37      13  0.23   
4570  28    02  2021         0        0  50           19      10  0.14   

      conversions  
0               0  
1               0  
2               0  
3               0  
4          

In [12]:
# Normalize the features
sscaler = StandardScaler()
cols = X_train.columns
X_train_ss = sscaler.fit_transform(X_train)
X_train_ss = pd.DataFrame(X_train_ss, columns = cols)
X_train_ss

,day,month,year,campaign,adgroup,ad,impressions,clicks,cost,conversions
0,-1.653649,0.103533,-0.663088,0.0,-1.328065,-1.291476,-0.346072,-0.372834,-0.189472,-0.256367
1,-1.653649,0.103533,-0.663088,0.0,-0.414508,-1.291476,-0.462041,-0.432742,-0.195043,-0.256367
2,-1.653649,0.103533,-0.663088,0.0,0.499049,-1.291476,-0.401535,-0.392804,-0.192258,-0.256367
3,-1.653649,0.103533,-0.663088,0.0,1.412606,-1.291476,-0.441873,-0.392804,-0.189472,-0.256367
4,-1.653649,0.103533,-0.663088,0.0,-1.328065,-0.814974,0.778328,0.825321,-0.105211,0.509425
...,...,...,...,...,...,...,...,...,...,...
4566,1.418954,-1.315509,1.508097,0.0,0.499049,0.874441,-0.371283,-0.372834,-0.190169,-0.256367
4567,1.418954,-1.315509,1.508097,0.0,1.412606,0.874441,-0.462041,-0.432742,-0.195043,-0.256367
4568,1.418954,-1.315509,1.508097,0.0,0.499049,0.917759,-0.386409,-0.352865,-0.186687,-0.064919
4569,1.418954,-1.315509,1.508097,0.0,-1.328065,0.917759,-0.280524,-0.302942,-0.179027,-0.256367


In [13]:
# Normalize the features for test data too
X_test = test_data
cols1 = X_test.columns
X_test_ss = sscaler.fit_transform(X_test)
X_test_ss = pd.DataFrame(X_test_ss, columns = cols1)
X_test_ss

,day,month,year,campaign,adgroup,ad,impressions,clicks,cost,conversions
0,-1.708476,0.0,0.0,0.0,-1.312635,-1.358759,0.533645,0.427218,0.336378,0.108337
1,-1.708476,0.0,0.0,0.0,0.555007,-1.358759,-0.447332,-0.392656,-0.357891,-0.447328
2,-1.708476,0.0,0.0,0.0,1.488828,-1.358759,-0.615782,-0.592085,-0.561092,-0.447328
3,-1.708476,0.0,0.0,0.0,-0.378814,-1.358759,-0.645509,-0.636402,-0.628825,-0.447328
4,-1.708476,0.0,0.0,0.0,-0.378814,-0.916502,-0.635600,-0.636402,-0.628825,-0.447328
...,...,...,...,...,...,...,...,...,...,...
313,1.488723,0.0,0.0,0.0,1.488828,-0.031987,-0.655418,-0.636402,-0.628825,-0.447328
314,1.488723,0.0,0.0,0.0,0.555007,-0.031987,-0.615782,-0.636402,-0.628825,-0.447328
315,1.488723,0.0,0.0,0.0,-1.312635,-0.031987,-0.417605,-0.414815,-0.408691,-0.447328
316,1.488723,0.0,0.0,0.0,0.555007,0.189142,-0.576147,-0.569926,-0.544158,-0.447328


BUILDING THE MODEL

In [14]:
# Create a function to output y_test
def score(model, title='Default'):
    model.fit(X_train_ss, y_train)
    y_test = model.predict(X_test_ss)
    return y_test

In [15]:
# Metrics that can be used - In case of using validation set
'''
print(math.sqrt(mean_squared_error(y_test, y_pred)), title)
print(confusion_matrix(y_train_test, y_pred))
accuracy = round(accuracy_score(y_test, y_pred), 5)
print('Accuracy for', title, ':', accuracy, '\n')'''

"\nprint(math.sqrt(mean_squared_error(y_test, y_pred)), title)\nprint(confusion_matrix(y_train_test, y_pred))\naccuracy = round(accuracy_score(y_test, y_pred), 5)\nprint('Accuracy for', title, ':', accuracy, '\n')"

In [16]:
# Types of models used for common ML Regression problems
model_01 = LinearRegression()
model_02 = Ridge()
model_03 = Lasso()
model_04 = DecisionTreeRegressor()
model_05 = RandomForestRegressor()

GETTING FINAL RESULTS

In [17]:
# Get y_test for all models
target_1 = score(model_01, 'linreg')
target_2 = score(model_02, 'ridge')
target_3 = score(model_03, 'lasso')
target_4 = score(model_04, 'dtr')
target_5 = score(model_05, 'rfr')

In [18]:
# Store the results in new dataframes under revenue

result_1 = pd.DataFrame(target_1)
result_1.index = X_test.index
result_1.columns = ["revenue"]

result_2 = pd.DataFrame(target_2)
result_2.index = X_test.index
result_2.columns = ["revenue"]

result_3 = pd.DataFrame(target_3)
result_3.index = X_test.index
result_3.columns = ["revenue"]

result_4 = pd.DataFrame(target_4)
result_4.index = X_test.index
result_4.columns = ["revenue"]

result_5 = pd.DataFrame(target_5)
result_5.index = X_test.index
result_5.columns = ["revenue"]

# In order to submit a file
# result.to_csv("name.csv", index = False)

In [19]:
result_1

,revenue
0,295.127156
1,-181.989616
2,-157.153330
3,-155.388784
4,-162.813645
...,...
313,-186.454315
314,-199.930228
315,-231.765379
316,-207.296936


In [20]:
result_2

,revenue
0,295.539075
1,-182.141224
2,-157.014139
3,-155.664183
4,-163.020612
...,...
313,-186.404852
314,-199.895463
315,-232.200794
316,-207.356363


In [21]:
result_3

,revenue
0,298.329932
1,-190.023652
2,-158.379632
3,-161.165321
4,-166.965392
...,...
313,-184.524514
314,-196.887933
315,-237.860830
316,-206.796635


In [22]:
result_4

,revenue
0,165.0
1,0.0
2,0.0
3,0.0
4,0.0
...,...
313,0.0
314,0.0
315,0.0
316,0.0


In [23]:
result_5

,revenue
0,482.0133
1,0.0000
2,0.0000
3,0.0000
4,0.0000
...,...
313,0.0000
314,0.0000
315,0.0000
316,0.0000
